<h1><b>Lab Session 3: Word Embeddings (Word2Vec)</b></h1>


* **CBOW** predicts a target word from surrounding context words.
* **Skip-gram** predicts surrounding context words from a target word (better for rare words).
* **Negative Sampling** replaces full softmax with a small binary classification task: push real (target, context) pairs up; push randomly sampled “negative” pairs down.

**Key-terms**
* Window size: how far to look around the center word.
* Embedding dim: size of vector.
* Min count: drop very rare words.
* Subsampling: downweight very frequent words (“the”, “of”…).

**Learning goals:**

* Understand the distributional hypothesis and why embeddings work.

* Explain Word2Vec (CBOW vs Skip-gram, window size, negative sampling).

* Train Word2Vec with a library (fast path).

* Implement a tiny from-scratch Skip-gram + Negative Sampling model.


**Required tools:** Python 3.9+, basic NumPy/PyTorch, basic linear algebra (dot product, cosine)

In [ ]:
!pip install numpy gensim torch matplotlib scikit-learn nltk


<h2><b>1. Warm-up</b></h2>

**Idea:** “You shall know a word by the company it keeps.”
Given a toy corpus, compute a co-occurrence matrix and show that cosine-similar words share contexts.

**Note:** This isn’t Word2Vec—just a warm-up to see context-based similarity.

In [ ]:
import numpy as np
from collections import defaultdict

corpus = [
    "king queen royal palace",
    "man woman boy girl",
    "paris france capital city",
    "rome italy capital city",
    "king man queen woman",
]

tokens = [w for line in corpus for w in line.split()]
vocab = sorted(set(tokens))
idx = {w:i for i,w in enumerate(vocab)}

window = 2
cooc = np.zeros((len(vocab), len(vocab)), dtype=np.float32)

for line in corpus:
    ws = line.split()
    for i, w in enumerate(ws):
        wi = idx[w]
        for j in range(max(0, i-window), min(len(ws), i+window+1)):
            if i == j: continue
            cooc[wi, idx[ws[j]]] += 1

def cosine(a,b):
    return a@b / (np.linalg.norm(a)*np.linalg.norm(b) + 1e-9)

def most_sim(word, k=3):
    i = idx[word]
    sims = [(vocab[j], float(cosine(cooc[i], cooc[j]))) for j in range(len(vocab)) if j != i]
    return sorted(sims, key=lambda x: x[1], reverse=True)[:k]

print("Similar to 'king':", most_sim("king"))


Similar to 'king': [('palace', 0.8660253594734205), ('woman', 0.6172134141557665), ('royal', 0.4714044890112376)]


<h2><b>2. Using Library for Word2Vec</b></h2>
Use gensim to train Word2Vec and query similar words.

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

raw_text = """
Paris is the capital of France. Rome is the capital of Italy.
The king and the queen visited the royal palace.
A boy and a girl met a man and a woman in the city.
"""
sentences = [simple_preprocess(s) for s in raw_text.strip().split("\n")]

# Skip-gram (sg=1); negative sampling (negative>0)
model = Word2Vec(sentences,
                 vector_size=50, window=2, min_count=1,
                 sg=1, negative=5, epochs=200, workers=1, seed=0)

print(model.wv.most_similar("king", topn=5))
print(model.wv.similarity("paris", "france"))

[('visited', 0.5827963352203369), ('the', 0.36465612053871155), ('city', 0.30424636602401733), ('royal', 0.24916714429855347), ('girl', 0.186227485537529)]
0.20128465


<h2>What’s happening:<h2>

gensim builds vocab, samples (target, context) pairs within the window, then trains embeddings using skip-gram + neg sampling. model.wv[...] gives vectors.

**Exercise A:** Change window and vector_size. Observe changes in neighbors for “king”, “paris”.

<h2><b>3. From Scratch: Skip-gram + Negative Sampling</b></h2>

**3.1 Data preparation:**
* Tokenize
* Build vocab & frequent-word sampling table,
* Generate training pairs (center → context) within window,
* Draw negatives from a unigram distribution^0.75 (as in the original paper).

In [ ]:
import torch, random, math
from collections import Counter

text = "king queen royal palace man woman boy girl paris france capital city rome italy capital city king man queen woman"
tokens = text.split()

# Build vocab
min_count = 1
cnt = Counter(tokens)
vocab = [w for w,c in cnt.items() if c >= min_count]
stoi = {w:i for i,w in enumerate(vocab)}
itos = {i:w for w,i in stoi.items()}
ids = [stoi[w] for w in tokens]

# Unigram^0.75 table for negative sampling
pow_freq = torch.tensor([cnt[itos[i]]**0.75 for i in range(len(vocab))], dtype=torch.float)
neg_dist = pow_freq / pow_freq.sum()

def generate_pairs(ids, window=2):
    pairs = []
    for i, center in enumerate(ids):
        left = max(0, i-window); right = min(len(ids), i+window+1)
        for j in range(left, right):
            if i==j: continue
            pairs.append((center, ids[j]))
    return pairs

pairs = generate_pairs(ids, window=2)
#print(f'Token:{tokens}[ID: {ids}]')
for t,i in zip(tokens,ids):
  print(f'Token:{t} [ID:{i}]')
print(f'Sample pairs')
#print("Sample pairs:", pairs[:6], f"(total {len(pairs)})")


Token:king [ID:0]
Token:queen [ID:1]
Token:royal [ID:2]
Token:palace [ID:3]
Token:man [ID:4]
Token:woman [ID:5]
Token:boy [ID:6]
Token:girl [ID:7]
Token:paris [ID:8]
Token:france [ID:9]
Token:capital [ID:10]
Token:city [ID:11]
Token:rome [ID:12]
Token:italy [ID:13]
Token:capital [ID:10]
Token:city [ID:11]
Token:king [ID:0]
Token:man [ID:4]
Token:queen [ID:1]
Token:woman [ID:5]
Sample pairs: [(0, 1), (0, 2), (1, 0), (1, 2), (1, 3), (2, 0)] (total 74)


**3.2 Model:**

We maintain two embedding matrices:
* *in_embed* for centers,
* *out_embed* for contexts.

**Loss per batch:**
Sum of log-sigmoid for positive pairs + log-sigmoid(−score) for negatives.

In [ ]:
class SkipGramNeg(torch.nn.Module):
    def __init__(self, vocab_size, dim):
        super().__init__()
        self.in_embed  = torch.nn.Embedding(vocab_size, dim)
        self.out_embed = torch.nn.Embedding(vocab_size, dim)
        torch.nn.init.uniform_(self.in_embed.weight,  -0.5/dim, 0.5/dim)
        torch.nn.init.zeros_(self.out_embed.weight)

    def forward(self, center_ids, pos_context_ids, neg_context_ids):
        # center: (B, D), pos: (B, D), neg: (B, K, D)
        center = self.in_embed(center_ids)                  # [B, D]
        pos    = self.out_embed(pos_context_ids)            # [B, D]
        neg    = self.out_embed(neg_context_ids)            # [B, K, D]

        # Positive scores: dot(center, pos)
        pos_score = torch.sum(center * pos, dim=1)          # [B]
        pos_loss  = torch.log(torch.sigmoid(pos_score) + 1e-9)

        # Negative scores: dot(center, neg) -> want them small
        neg_score = torch.bmm(neg, center.unsqueeze(2)).squeeze(2)  # [B, K]
        neg_loss  = torch.log(torch.sigmoid(-neg_score) + 1e-9).sum(1)  # [B]

        # Maximize pos_loss + neg_loss -> minimize negative
        loss = -(pos_loss + neg_loss).mean()
        return loss

**3.3 Training loop**

* For each (center, positive_context), sample K negatives from neg_dist (avoiding the positive).
* Optimize with Adam for a few epochs.

In [ ]:
device = "cpu"
dim = 50
K = 5
batch_size = 32
epochs = 200
model = SkipGramNeg(len(vocab), dim).to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.01)

def sample_neg(batch_size, K):
    # Multinomial draws negatives i.i.d. from neg_dist
    return torch.multinomial(neg_dist, batch_size*K, replacement=True).view(batch_size, K)

# Tiny dataset: convert pairs to tensors
pairs_t = [(torch.tensor(c), torch.tensor(p)) for c,p in pairs]

for ep in range(1, epochs+1):
    random.shuffle(pairs_t)
    total = 0.0
    for i in range(0, len(pairs_t), batch_size):
        batch = pairs_t[i:i+batch_size]
        c_ids = torch.stack([b[0] for b in batch]).to(device)
        p_ids = torch.stack([b[1] for b in batch]).to(device)
        n_ids = sample_neg(len(batch), K).to(device)

        loss = model(c_ids, p_ids, n_ids)
        opt.zero_grad(); loss.backward(); opt.step()
        total += float(loss)

    if ep % 50 == 0:
        print(f"epoch {ep}: loss={total/ (len(pairs_t)/batch_size):.4f}")

# Get final input embeddings
emb = model.in_embed.weight.detach().cpu().numpy()

/tmp/ipython-input-2595951285.py:27: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  total += float(loss)


epoch 50: loss=2.6042
epoch 100: loss=2.4678
epoch 150: loss=2.3446
epoch 200: loss=2.2777


**What the code does (in one epoch):**

We build training pairs using a sliding window; for each positive pair we draw K negatives; the model learns two embedding tables so that dot products of true (center, context) go high, and with random contexts go low. The objective is the sum of log-sigmoid terms.

**3.4 Inspect embeddings**

Nearest neighbors by cosine similarity:

In [ ]:
from numpy.linalg import norm

def most_sim(word, topk=5):
    wid = stoi[word]
    v = emb[wid]
    sims = []
    for i in range(len(vocab)):
        if i == wid: continue
        s = float(v @ emb[i] / (norm(v)*norm(emb[i]) + 1e-9))
        sims.append((itos[i], s))
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[:topk]

print("NN(king):", most_sim("king"))
print("NN(paris):", most_sim("paris"))


NN(king): [('palace', 0.6836001783374662), ('rome', 0.5998950395818237), ('italy', 0.48061624502538514), ('royal', 0.4591892446706788), ('france', 0.3587692084619158)]
NN(paris): [('girl', 0.6003517195244562), ('france', 0.5463207315231119), ('woman', 0.5261117139363036), ('italy', 0.4479966605957958), ('city', 0.40906032642926526)]


In [ ]:
def analogy(a,b,c, topk=3):
    va, vb, vc = emb[stoi[a]], emb[stoi[b]], emb[stoi[c]]
    q = vb - va + vc
    sims = []
    for i in range(len(vocab)):
        w = itos[i]
        if w in {a,b,c}: continue
        s = float(q @ emb[i] / (norm(q)*norm(emb[i]) + 1e-9))
        sims.append((w,s))
    sims.sort(key=lambda x: x[1], reverse=True)
    return sims[:topk]

print("king:man :: ? : woman →", analogy("king","man","woman"))


king:man :: ? : woman → [('girl', 0.5089562247328274), ('royal', 0.49312329477134664), ('paris', 0.36166756567992847)]


<h2><b>4. Word2Vec with gensim on 20 Newsgroups (≈11k documents)</b></h2>

* Load a real-world text dataset (20 Newsgroups).
* Train skip-gram + negative sampling Word2Vec in gensim.
* Explore nearest neighbors, word similarity, and simple analogies.
* Save & reload the model.

In [ ]:
import nltk; nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

**4.1 Load & preprocess the dataset**

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

# Grab a reasonable slice (you can use subset='all' if you like)
data = fetch_20newsgroups(subset='train', remove=('headers','footers','quotes'))
docs = data.data  # ~11k short news-like posts

# Convert documents → list of tokenized sentences for Word2Vec
# sent_tokenize splits into sentences (better training pairs).
# simple_preprocess = lowercase + basic cleanup. No heavy NLP pipeline needed.
sentences = []
for doc in docs:
    for s in sent_tokenize(doc):
        tokens = simple_preprocess(s, min_len=2, max_len=20)  # lowercase, strip punct, keep words
        if tokens:
            sentences.append(tokens)

print(len(docs), "documents")
print(docs[0])
print(len(sentences), "sentences, e.g.:", sentences[0][:12])

11314 documents
I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.
125218 sentences, e.g.: ['was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car']


**4.2 Train Word2Vec (skip-gram + negative sampling)**

**Tip:** If many probes are “not in vocab,” lower min_count to 3 (slower, bigger vocab) or raise epochs.

In [ ]:
from gensim.models import Word2Vec

# Small, fast settings for a demo. You can increase vector_size/epochs later for better quality.
w2v = Word2Vec(
    sentences,
    vector_size=100,      # embedding dim
    window=5,             # context window
    min_count=5,          # ignore very rare words
    workers=4,            # CPU threads
    sg=1,                 # 1=skip-gram, 0=CBOW
    negative=10,          # negative samples
    epochs=5,             # training passes (raise to 10–20 for quality)
    sample=1e-3,          # subsampling for frequent words
    seed=42
)

**What this does (brief):**

Skip-gram creates (center → context) pairs inside a sliding window and learns vectors so true context pairs score high and random pairs score low (negative sampling).

<h2><b>Short exercises (Make changes to model)</b></h2>

1. Window size study: Train with window ∈ {1,2,5}. Report nearest neighbors of capital and king. What changes? Why?

2. Negative samples K: Try K ∈ {2,5,15}. How do loss and neighbors change?

3. Rare words: Add new sentences that use palace and royal more often. Does “palace” move closer to “royal/king/queen”?

4. CBOW vs Skip-gram (challenge): Modify the training pairs so context→target (average context embeddings to predict center). Compare.

5. Subsampling (extra): Downsample frequent tokens (e.g., “the”, “is”). Observe speed/quality.

**4.3 Quick exploration**

In [ ]:
wv = w2v.wv  # keyed vectors

def try_neighbors(word, topn=10):
    if word in wv:
        print(f"\nMost similar to '{word}':")
        for w,score in wv.most_similar(word, topn=topn):
            print(f"  {w:15s}  {score:.3f}")
    else:
        print(f"'{word}' not in vocab (try a more frequent term).")

# Try domain-relevant words commonly found in 20NG:
for probe in ["computer", "windows", "mac", "graphics", "game",
              "religion", "god", "church", "hockey", "space",
              "science", "medical", "gun", "government"]:
    try_neighbors(probe)



Most similar to 'computer':
  engineering      0.695
  bulletin         0.691
  shopper          0.690
  electronic       0.686
  vlsi             0.680
  peripheral       0.674
  calculator       0.674
  silicon          0.665
  isdn             0.664
  architecture     0.661

Most similar to 'windows':
  apps             0.820
  microsoft        0.804
  workgroups       0.783
  excel            0.779
  emm              0.753
  os               0.750
  compiler         0.750
  paradox          0.750
  povray           0.750
  sdk              0.749

Most similar to 'mac':
  amiga            0.769
  atari            0.758
  clone            0.755
  iisi             0.751
  roms             0.738
  keyboards        0.734
  netware          0.734
  iici             0.734
  trident          0.733
  compaq           0.733

Most similar to 'graphics':
  gems             0.761
  animation        0.761
  assembler        0.753
  multimedia       0.753
  raytracing       0.751
  cad          

**4.4 Word similarity & analogies**

In [ ]:
def sim(a, b):
    if a in wv and b in wv:
        print(f"similarity({a}, {b}) = {wv.similarity(a, b):.3f}")
    else:
        print(f"Missing: {a if a not in wv else ''} {b if b not in wv else ''}")

# Similarity checks
sim("computer", "graphics")
sim("hockey", "game")
sim("religion", "church")
sim("space", "nasa")

# Analogy: a : b :: c : ?
# Example: "king - man + woman ≈ queen" often won’t work perfectly on this dataset—try topical ones:
def analogy(a, b, c, topn=5):
    if all(w in wv for w in [a,b,c]):
        print(f"\nAnalogy: {a} : {b} :: {c} : ?")
        for w,score in wv.most_similar(positive=[b, c], negative=[a], topn=topn):
            print(f"  {w:15s}  {score:.3f}")
    else:
        print("Analogy words missing from vocab.")

analogy("computer", "windows", "mac")     # OS/software vibe
analogy("space", "nasa", "astronomy")     # topical
analogy("hockey", "game", "team")


similarity(computer, graphics) = 0.503
similarity(hockey, game) = 0.635
similarity(religion, church) = 0.700
similarity(space, nasa) = 0.626

Analogy: computer : windows :: mac : ?
  os               0.689
  apps             0.651
  nt               0.629
  linux            0.614
  exe              0.611

Analogy: space : nasa :: astronomy : ?
  jpl              0.818
  dryden           0.751
  larc             0.747
  lerc             0.747
  spacelink        0.744

Analogy: hockey : game :: team : ?
  scored           0.672
  score            0.668
  games            0.665
  inning           0.659
  wins             0.645
